<a href="https://colab.research.google.com/github/WalrusEggMan/intro-to-is/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/WalrusEggMan/intro-to-is/master/data.csv'

data = pd.read_csv(url, delimiter=',')
data.head()


# New Section